# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4941, uptime 0:00:34
mini_beamline                    RUNNING   pid 4942, uptime 0:00:34
random_walk                      RUNNING   pid 4943, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4944, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4945, uptime 0:00:34
simple                           RUNNING   pid 4946, uptime 0:00:34
thermo_sim                       RUNNING   pid 4947, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4948, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-07 20:24:40
Persistent Unique Scan ID: '55ffd6c8-ee78-4914-a90c-a2e5d18e9c37'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:24:40.1 |     -1.000 |      97317 |
|         2 | 20:24:40.1 |     -0.500 |      99099 |
|         3 | 20:24:40.1 |      0.000 |     100380 |
|         4 | 20:24:40.2 |      0.500 |      99868 |
|         5 | 20:24:40.2 |      1.000 |      98430 |


+-----------+------------+------------+------------+
generator scan ['55ffd6c8'] (scan num: 3)





('55ffd6c8-ee78-4914-a90c-a2e5d18e9c37',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/55ffd6c8-ee78-4914-a90c-a2e5d18e9c37-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-07 20:24:40
Persistent Unique Scan ID: '81e2f7a4-8194-4db1-9e9b-dbbfe412c384'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:24:40.6 |          -1 | 1108970178 |


|         2 | 20:24:40.8 |          -0 | 1117657598 |


|         3 | 20:24:40.9 |           0 | 1125548759 |


|         4 | 20:24:41.0 |           0 | 1129077552 |


|         5 | 20:24:41.1 |           1 | 1130941773 |
+-----------+------------+-------------+------------+
generator scan ['81e2f7a4'] (scan num: 4)





('81e2f7a4-8194-4db1-9e9b-dbbfe412c384',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-07 20:24:41
Persistent Unique Scan ID: '334968a8-127b-41a2-9c43-bacf7520055f'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:24:41.6 | 1128350373 |


|         2 | 20:24:42.5 | 1076051235 |


|         3 | 20:24:43.5 | 1056955107 |


...

|        28 | 20:25:08.5 | 1111523006 |


|        29 | 20:25:09.5 | 1126755016 |


|        30 | 20:25:10.5 | 1073022600 |


+-----------+------------+------------+
generator count ['334968a8'] (scan num: 5)





('334968a8-127b-41a2-9c43-bacf7520055f',)